In [1]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

import pandas as pd
from pandas.io.json import json_normalize

In [2]:
authenticator = IAMAuthenticator('bkC6hyTVS4qonY34vS0d8Q5F2YY4Kz9m0fyj1cN-5VFw')

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze?version=2019-07-12')

text = """
我是從MRT Bayfront站, 行到Garden by the Bay 濱海灣花園的, 天氣炎熱, 有點辛苦, 建議可考慮坐計程車前往. 公園好大, 景點分別有擎天大樹林, 雲霧森林玻璃屋 Cloud Forest及花穹Flower Dome, 各具特色, 雖然要付費, 都好值得一看.
"""

response = natural_language_understanding.analyze(
    text=text,
    features=Features(entities=EntitiesOptions(emotion=True, sentiment=True, limit=100000), keywords=KeywordsOptions(emotion=True, sentiment=True, limit=100000))).get_result()

print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 147,
    "features": 1
  },
  "language": "zh",
  "keywords": [
    {
      "text": "\u516c\u5712\u597d\u5927",
      "sentiment": {
        "score": 0.881923,
        "label": "positive"
      },
      "relevance": 0.915041,
      "count": 1
    },
    {
      "text": "\u666f\u9ede\u5206\u5225",
      "sentiment": {
        "score": 0.881923,
        "label": "positive"
      },
      "relevance": 0.695366,
      "count": 1
    },
    {
      "text": "\u6ff1\u6d77\u7063\u82b1\u5712",
      "sentiment": {
        "score": 0.703319,
        "label": "positive"
      },
      "relevance": 0.675763,
      "count": 1
    },
    {
      "text": "\u96f2\u9727\u68ee\u6797\u73bb\u7483\u5c4b",
      "sentiment": {
        "score": 0.881923,
        "label": "positive"
      },
      "relevance": 0.623137,
      "count": 1
    },
    {
      "text": "\u5927\u6a39\u6797",
      "sentiment": {
        "score": 0.881923,
        "label": "p

In [9]:
keywords_col_names = ['WEBSITE_ID', 'REVIEW_ID', 'TEXT', 'RELEVANCE', 'COUNT',
                     'SENTIMENT_SCORE', 'SENTIMENT_LABEL', 'SADNESS', 'JOY',
                     'FEAR', 'DISGUST', 'ANGER', 'MIXED_SENTIMENT']

keywords_col_names_map = {
    'text': 'TEXT', 
    'relevance': 'RELEVANCE', 
    'count': 'COUNT', 
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT'
}

keywords_col_names_df = pd.DataFrame(columns=keywords_col_names)

keywords_df = json_normalize(response['keywords']).rename(columns=keywords_col_names_map)
keywords_df.insert(0, 'REVIEW_ID', 123456)
keywords_df.insert(0, 'WEBSITE_ID', 1)
keywords_df = keywords_col_names_df.append(keywords_df, sort=False)
keywords_df


,WEBSITE_ID,REVIEW_ID,TEXT,RELEVANCE,COUNT,SENTIMENT_SCORE,SENTIMENT_LABEL,SADNESS,JOY,FEAR,DISGUST,ANGER,MIXED_SENTIMENT
0,1,123456,公園好大,0.915041,1,0.881923,positive,NaN,NaN,NaN,NaN,NaN,NaN
1,1,123456,景點分別,0.695366,1,0.881923,positive,NaN,NaN,NaN,NaN,NaN,NaN
2,1,123456,濱海灣花園,0.675763,1,0.703319,positive,NaN,NaN,NaN,NaN,NaN,NaN
3,1,123456,雲霧森林玻璃屋,0.623137,1,0.881923,positive,NaN,NaN,NaN,NaN,NaN,NaN
4,1,123456,大樹林,0.612947,1,0.881923,positive,NaN,NaN,NaN,NaN,NaN,NaN
5,1,123456,花穹,0.491022,1,0.881923,positive,NaN,NaN,NaN,NaN,NaN,NaN
6,1,123456,是從,0.383536,1,0.703319,positive,NaN,NaN,NaN,NaN,NaN,NaN
7,1,123456,天氣,0.305391,1,0.703319,positive,NaN,NaN,NaN,NaN,NaN,NaN
8,1,123456,建議,0.295996,1,0.703319,positive,NaN,NaN,NaN,NaN,NaN,NaN
9,1,123456,考慮,0.291341,1,0.703319,positive,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
entities_col_names = ['WEBSITE_ID', 'REVIEW_ID', 'TYPE', 'TEXT', 'RELEVANCE',
                      'COUNT', 'CONFIDENCE', 'SENTIMENT_SCORE',
                      'SENTIMENT_LABEL', 'SADNESS', 'JOY', 'FEAR', 'DISGUST',
                      'ANGER', 'MIXED_SENTIMENT', 'DISAMBIGUATION_SUBTYPE', 'DISAMBIGUATION_NAME', 
                      'DISAMBIGUATION_RESOURCE']

entities_col_names_map = {
    'type': 'TYPE',
    'text': 'TEXT', 
    'relevance': 'RELEVANCE', 
    'count': 'COUNT',
    'confidence': 'CONFIDENCE',
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT',
    'disambiguation.subtype': 'DISAMBIGUATION_SUBTYPE',
    'disambiguation.name': 'DISAMBIGUATION_NAME',
    'disambiguation.dbpedia_resource': 'DISAMBIGUATION_RESOURCE'
}

if 'entities' in response:
    entities_df = json_normalize(response['entities']).rename(columns=entities_col_names_map)
    entities_df.insert(0, 'REVIEW_ID', 1234567890)
    entities_df.insert(0, 'WEBSITE_ID', 1)
    entities_df.shape

In [ ]:
entities_df.columns.values